<a name="1common."></a>
## Содержание:
* [1. Описание проекта, описание данных, загрузка и импорт библиотек](#1.)
* [2. Предобработка данных](#2.)
     - [2.1. Обработка пропусков](#2.1.)
     - [2.2. Замена типа данных](#2.2.)
     - [2.3. Обработка дубликатов](#2.3.)
     - [2.4. Лемматизация](#2.4.)
* [3. Зависимость между категориями заемщиков](#3.)
* [4. Общий вывод](#4.)

 <a name="1."><a/><br/>
    <font size ="5"><b>1. Описание проекта, описание данных, загрузка и импорт библиотек</b></font><br/>
[<font size="2">(к содержанию)</font>](#1common.)

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

In [12]:
# для работы с таблицами
import pandas as pd

# для лемматизации
from pymystem3 import Mystem
from collections import Counter

In [7]:
df = pd.read_csv('/datasets/data.csv')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


#### Данные не удобны для изучения:

Трудовой стаж - минусы и нецелочисленные цифры. Информация записана очень некорректно, скорее всего данный столбец не поможет в анализе. 

Образование - разный шрифт.

Ежемесячный доход - слишком много знаков после запятой.

Вывод: привести данные к удобному для анализа виду.

Пропущены значения в столбцах 'days_employed' и 'total_income'. Так же в этих столбцах тип данных 'строка', а для удобства проведения аналитических операций нужно преобразовать в 'int'

<a name="2."><a/><br/>
    <font size ="5"><b>2.Предобработка данных</b></font><br/>
[<font size="2">(к содержанию)</font>](#1common.)
    <a name="2.1."><a/><br/>
    <font size ="4"><b>2.1.Обработка пропусков</b></font><br/>
[<font size="2">(к содержанию)</font>](#1common.)

In [8]:
print(df.isna().sum())
df = df.fillna(0)
df.info()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Пропущены значения в количестве отработанных дней и в зарплате.

Можно сделать вывод о том, что нету ни работы, ни зарплаты)

Заменили пропущенные значения на "0". 0 дней работы, 0 рублей)

<a name="2.2."><a/><br/>
    <font size ="4"><b>2.2.Замена типов данных</b></font><br/>
[<font size="2">(к содержанию)</font>](#1common.)

In [9]:
# применен самый простой метод сменны типа данных astype, так как он меняет весь массив
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

# проверяем данные
df.tail()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Меняем типы данных в столбце days_employed и total_income на целые числа. Использован метод замены типа данных всего столбца, для удобства восприятия цифр.

<a name="2.3."><a/><br/>
    <font size ="4"><b>2.3.Обработка дубликатов</b></font><br/>
[<font size="2">(к содержанию)</font>](#1common.)

In [10]:
# строки в столбце education приведенны к одному виду
df['education'] = df['education'].str.lower()

# использован метод ручного поиска дубликатов, так как все строки в одном регистре
df.duplicated().sum()

# Удаленны повторяющиеся строки
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

### Вывод

Появление дубликатов связанно скорей всего с человеческой ошибкой (клиент обратился дважды, и вместо использовании первой анкеты, была добавлена вторая, идентичная) 

<a name="2.4."><a/><br/>
    <font size ="4"><b>2.4.Лемматизация</b></font><br/>
[<font size="2">(к содержанию)</font>](#1common.)

In [13]:
# выделил уникальные значения в столбце в массив
uni = df['purpose'].unique()
text = ' '.join(uni)

# провел лемматизацию и подсчет словоформ
m = Mystem()
lemmas = m.lemmatize(text)
Counter(lemmas)

Counter({'покупка': 10,
         ' ': 96,
         'жилье': 7,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

### Вывод

Много повторяющихся по смыслу слов. покупка = приобретение, сыграть = провести

<a name="2.5."><a/><br/>
    <font size ="4"><b>2.5. Категоризация данных</b></font><br/>
[<font size="2">(к содержанию)</font>](#1common.)

In [14]:
# подсчитал квантили(нижний, средний, верхний), для нахождения диапазона среднего дохода
v = df['total_income'].quantile(0.25)
c = df['total_income'].quantile(0.5)
d = df['total_income'].quantile(0.75)
v, c, d

(89088.5, 135781.0, 195813.25)

In [15]:
# функция для категоризации уровня дохода
def money(total_income):
    if total_income > 88612 and total_income <= 195543:
        return 'Средний доход'
    if total_income > 195543:
        return 'Высокий доход'
    return 'Низкий доход'


# новый столбец уровня дохода
df['money'] = df['total_income'].apply(money)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,money
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,Высокий доход
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,Средний доход
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,Средний доход
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,Высокий доход
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,Средний доход


In [16]:
# функция для лемматизации столбца
def lemmarow(row):
    word = row['purpose']
    lem = ' '.join(m.lemmatize(word))
    return lem


# новый, лемматизированный столбец
df['general_purpose'] = df.apply(lemmarow, axis=1)

In [17]:
# функция для категоризации целей кредита
def aim(row):
    if 'жилье' in row:
        return 'Недвижимость'
    if 'недвижимость' in row:
        return 'Недвижимость'
    if 'автомобиль' in row:
        return 'Автомобиль'
    if 'свадьба' in row:
        return 'Свадьба'
    if 'образование' in row:
        return 'Учеба'


aim(df['general_purpose'])
df['aim'] = df['general_purpose'].apply(aim)
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,money,general_purpose,aim
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,Высокий доход,покупка жилье \n,Недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,Средний доход,приобретение автомобиль \n,Автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,Средний доход,покупка жилье \n,Недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,Высокий доход,дополнительный образование \n,Учеба
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,Средний доход,сыграть свадьба \n,Свадьба


In [18]:
# проверил отсутствие промущенных значений в новом столбце категорий кредита
df['aim'].isna().sum()

0

### Вывод

#### 4 основных категории целей заема, 3 уровня дохода

 <a name="3."><a/><br/>
    <font size ="5"><b>3. Зависимость между категориями заемщиков</b></font><br/>
[<font size="2">(к содержанию)</font>](#1common.)

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [19]:
# найдем часть невернувжих кредит
gf = df.groupby('children').agg({'debt': ['count', 'sum']})
gf['part'] = (gf['debt']['sum'] / gf['debt']['count']) * 100

# округлим для наглядности
gf['part'] = gf['part'].round(1)
gf.sort_values(by='part', ascending=False)

debt        part
          count   sum      
children                   
 20          76     8  10.5
 4           41     4   9.8
 2         2052   194   9.5
 1         4808   444   9.2
 3          330    27   8.2
 0        14091  1063   7.5
-1           47     1   2.1
 5            9     0   0.0

### Вывод

Замечены странные данные '-1' и '20'. Что это за ошибка - непонятно. Назовем их 'выбросами' и не будем обращать внимание. так как их наличие не существенно. 

Без детей просрочек меньше.
Скорей всего связанно с тем, что с детьми сложнее планировать бюджет. 
Вывод: дети не залог здорового финансового состояния.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [13]:
# count - всего должников, sum - невернувших
gf1 = df.groupby('family_status').agg({'debt': ['count', 'sum']})

# доля невернувших
gf1['part'] = ((gf1['debt']['sum'] / gf1['debt']['count']) * 100).round(1)
gf1.sort_values(by='part', ascending=False)

debt      part
                       count  sum     
family_status                         
Не женат / не замужем   2810  274  9.8
гражданский брак        4151  388  9.3
женат / замужем        12339  931  7.5
в разводе               1195   85  7.1
вдовец / вдова           959   63  6.6

### Вывод

Самые своевременные женатые и вдовцы. Люди умеют брать ответственность на себя. 

Даем кредиты им

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [14]:
# count - всего должников, sum - невернувших
gf2 = df.groupby('money').agg({'debt': ['count', 'sum']})

# доля невернувших
gf2['part'] = ((gf2['debt']['sum'] / gf2['debt']['count']) * 100).round(1)
gf2

debt      part
               count  sum     
money                         
Высокий доход   5381  386  7.2
Низкий доход    5311  417  7.9
Средний доход  10762  938  8.7

### Вывод

Доход не имеет особого влияние на просрочку кредита

- Как разные цели кредита влияют на его возврат в срок?

In [15]:
# count - всего должников, sum - невернувших
gf3 = df.groupby('aim').agg({'debt': ['count', 'sum']})

# доля невернувших
gf3['part'] = ((gf3['debt']['sum'] / gf3['debt']['count']) * 100).round(1)
gf3.sort_values(by='part', ascending=False)

debt      part
              count  sum     
aim                          
Автомобиль     4306  403  9.4
Учеба          4013  370  9.2
Свадьба        2324  186  8.0
Недвижимость  10811  782  7.2

### Вывод

На недвижимость - без просрочек, а на долги по учебе и за машину можно и забить))

Видимо связанно с тем, что люди, приобретающие жилье более ответственные. Или давят залоговые обязательства.

 <a name="4."><a/><br/>
    <font size ="5"><b>4. Общий вывод</b></font><br/>
[<font size="2">(к содержанию)</font>](#1common.)


    
    Зависимость по количеству детей:(доля просрочек указана в процентах)
    
    1 ребенок - 9.2% 
    2 ребенка - 9.4%
    3 ребенка - 8.2%
    Без детей - 7.5%
    Прямой зависимости нету, но наличие детей явно влияет на просрочки
    
    Зависимость по семейному положению:
    
    Женатые - 7.5%
    Холостые - 9.7%
    Семейные люди более ответственные
   
    Зависимость по доходу:
    
    Низкий доход - 7.7%
    Средний доход - 8.7%
    Высокий доход - 7.2%
    Люди со среднем доходом, категория риска
    
    Зависимость по целям:
    
    Автомобиль - 9.3%
    Учеба - 9.2%
    Свадьба	- 7.9%
    Недвижимость - 7.2%
    Люди, берущие кредиты на свадьбу и недвижимость самые ответственные
